# 一致预期评级表

# R03

In [1]:

class MysqlConn(object):
    def __init__(self, config_json_str):
        # self.mysql_config = {"host": "10.22.128.150",  "port": 3317, "db": "bigdata", "user": "talend_load",
        #   "passwd": "s9t5gNThn2vqWM7c" , "charset" : "utf8"}
        # self.mysql_config = {"host": "db-bigdata.wmcloud-qa.com",  "port": 3312, "db": "bigdata", "user": "app_bigdata_ro",
        #   "passwd": "Welcome_20141217"}
        self.mysql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mysql_conn = pymysql.connect(**self.mysql_config)
        return mysql_conn

class MssqlConn(object):
    def __init__(self, config_json_str):
        # self.mssql_config = {"server": "sh-datayesdb.wmcloud-dev.com",  "port": 1433, "database": "datayesdb",
        #     "user": "talend_load", "password": "Welcome01"}
        self.mssql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mssql_conn = pymssql.connect(**self.mssql_config)
        return mssql_conn

    def __str__(self):
        return json.dumps(self.mssql_config, ensure_ascii=False, encoding='utf-8')



In [2]:
import json
import pymysql
import pymssql
# MySQL
# bigdata={"host":"security03-dev.datayes.com","port":3306,"user":"talend_load","passwd":"NCph1G9BQT3DuQj","db":"bigdata","charset":"utf8"}
# dateyesdbp={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"user":"app_dataqa_ro","passwd":"Welcome20140820","db":"datayesdbp","charset":"utf8"}
# datayesdb={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"database":"datayesdb","user":"app_gaea_ro","password":"EQw6WquhnCKPp8Li","charset":"utf8"}

# researchrpt={"host":"db-researchrpt-ro.wmcloud.com","port":3306,"user":"app_gaea_ro","passwd":"UDVVkvULzakP2048","db":"research_rpt","charset":"utf8"}
# dev
# researchrpt={"host":"10.24.21.34","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8"}
# prd
researchrpt={"host":"db-researchrpt.wmcloud.com","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8"}

# SQLserver
# datayes={"server":"sh-dm-db05.datayes.com","port":1433,"database":"dyedb","user":"talend_load","password":"s9t5gNThn2vqWM7c","charset":"utf8"}



# bigdata_conn = MysqlConn(json.dumps(bigdata)).connect()
# dateyesdbp_conn = MysqlConn(json.dumps(dateyesdbp)).connect()
# datayesdb_conn = MysqlConn(json.dumps(datayesdb)).connect()
researchrpt_conn = MysqlConn(json.dumps(researchrpt)).connect()

# datayes_conn = MssqlConn(json.dumps(datayes)).connect()


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [81]:
# base_df
filter_str = "CREATE_DATE" # 根据 num 列分开1，2 后， 需要从num=1的数据中筛选出日期为查询日期的数据
this_date_str = '2019-01-22'
primary_key = ["CODE", "ORGAN_ID"] # 用于merge
last_key = ["CREATE_DATE", "SCORE_ID"] # 同一个机构上次对本证券做的预测报告的撰写日期 同一个机构上次对本证券做的预测报告的评级

In [82]:
sql = """

SELECT ORIGIN_ID, CODE, CODE_NAME, TITLE, AUTHOR, CREATE_DATE, SCORE_ID, ORGAN_ID, num
FROM (
	SELECT ORIGIN_ID, CODE, CODE_NAME, TITLE, AUTHOR, CREATE_DATE, SCORE_ID, ORGAN_ID,
		   (@row_number:=
			   case when CODE = @pre_code and ORGAN_ID = @pre_organ_id
			then @row_number+1
			 else 1
			end
		   ) as num,
		(@pre_code := CODE),
		(@pre_organ_id := ORGAN_ID)
	FROM cmb_report_research,(select @row_number:=1,@pre_code:='',@pre_organ_id:='') a
	where CREATE_DATE <= '%s 00:00:00'
	order by CODE,ORGAN_ID,CREATE_DATE desc
) a
where a.num<=2
""" % (this_date_str)
base_df = pd.read_sql(sql, researchrpt_conn)

In [83]:
base_df['CODE'].replace(u'', np.nan, inplace=True)
base_df['ORGAN_ID'].replace(u'', np.nan, inplace=True)
base_df = base_df.dropna(subset=primary_key, how='any')

In [84]:
base_df.iloc[0].values[0]

2264832

In [85]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34482 entries, 18 to 34499
Data columns (total 9 columns):
ORIGIN_ID      34482 non-null int64
CODE           34482 non-null object
CODE_NAME      34482 non-null object
TITLE          34482 non-null object
AUTHOR         34416 non-null object
CREATE_DATE    34482 non-null datetime64[ns]
SCORE_ID       27753 non-null float64
ORGAN_ID       34482 non-null int64
num            34482 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 2.6+ MB


In [86]:
# 用num的值区分次序
base_df["num"] = base_df["num"].astype(int)
last_df = base_df[base_df["num"]==2]
last_df = last_df.drop("num", 1)

In [87]:
last_df.head()
# last_df.loc[1, 'CODE']

,ORIGIN_ID,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,SCORE_ID,ORGAN_ID
19,2258023,000001,平安银行,11月策略：重点推荐兴业银行,"邱冠华,袁梓芳,梁凤洁",2018-11-05,7.0,2
21,2255287,000001,平安银行,宁波银行持续领跑；关注贵阳银行资产质量,张帅帅,2018-10-30,0.0,3
24,2262895,000001,平安银行,经营平稳，收获价值,肖斐斐,2018-11-26,0.0,13
26,2141802,000001,平安银行,"银行业：农行定增落地且获政策支持,“一委一行两会”监管框架形成",杨荣,2018-03-19,7.0,20
28,2154763,000001,平安银行,"银行1季报前瞻：息差与资产质量稳步改善,规模增速低位运行","励雅敏,袁喆奇",2018-04-18,5.0,22


In [88]:
# last_df[['ORIGIN_ID']]

In [89]:
base_df[filter_str] = base_df[filter_str].astype(str)

this_df = base_df[(base_df["num"]==1) & (base_df[filter_str]==this_date_str)]
this_df = this_df.drop("num", 1)


In [90]:
if len(this_df) == 0:
    # 说明没有目标数据， 可以返回
    pass


In [91]:
this_df.head()

,ORIGIN_ID,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE,SCORE_ID,ORGAN_ID
31,3616257,000001,平安银行,四季度基金持仓分析：整体持仓小幅下滑，配置偏好业绩改善个股,王一川,2019-01-22,5.0,28
52,3602608,000002,万科A,地产12月观察：主要矛盾，和新的超预期,"谢皓宇,卜文凯,白淑媛",2019-01-22,7.0,2
64,3604831,000002,万科A,房地产行业2019年1月跟踪报告：难得的错配,"陈聪,张全国",2019-01-22,7.0,13
87,3604754,000002,万科A,2018年房地产投资销售保持高位运行,"高建,张云凯",2019-01-22,5.0,119
123,3604916,000012,南玻A,建材行业周报：水泥行情望保持高位，关注基建补短板启动,中银国际研究所,2019-01-22,5.0,22


In [92]:
key = last_key + primary_key


In [93]:
# merge_df = pd.merge(this_df, last_df[key], how='right', on=primary_key)

In [94]:

merge_df = pd.merge(this_df, last_df[key], how='left', on=primary_key)

In [95]:
merge_df = merge_df.dropna(subset=["CREATE_DATE_y"])

In [96]:
merge_df.head()

,ORIGIN_ID,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE_x,SCORE_ID_x,ORGAN_ID,CREATE_DATE_y,SCORE_ID_y
0,3616257,000001,平安银行,四季度基金持仓分析：整体持仓小幅下滑，配置偏好业绩改善个股,王一川,2019-01-22,5.0,28,2019-01-19,5.0
1,3602608,000002,万科A,地产12月观察：主要矛盾，和新的超预期,"谢皓宇,卜文凯,白淑媛",2019-01-22,7.0,2,2019-01-13,7.0
2,3604831,000002,万科A,房地产行业2019年1月跟踪报告：难得的错配,"陈聪,张全国",2019-01-22,7.0,13,2019-01-03,7.0
3,3604754,000002,万科A,2018年房地产投资销售保持高位运行,"高建,张云凯",2019-01-22,5.0,119,2019-01-21,5.0
4,3604916,000012,南玻A,建材行业周报：水泥行情望保持高位，关注基建补短板启动,中银国际研究所,2019-01-22,5.0,22,2019-01-16,5.0


In [97]:
# merge 获取上一次最近的数据时， 若上一次数据为空，` 删除它们
# merge_df = merge_df.dropna(subset=['WRITE_DATE_y'])
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 410 entries, 0 to 417
Data columns (total 10 columns):
ORIGIN_ID        410 non-null int64
CODE             410 non-null object
CODE_NAME        410 non-null object
TITLE            410 non-null object
AUTHOR           410 non-null object
CREATE_DATE_x    410 non-null object
SCORE_ID_x       410 non-null float64
ORGAN_ID         410 non-null int64
CREATE_DATE_y    410 non-null datetime64[ns]
SCORE_ID_y       409 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 35.2+ KB


In [98]:
# cmb_report_research.ORGAN_ID=report_org_map.TGT_ORG_ID
sql = """SELECT distinct TGT_ORG_ID,TGT_ORG_NAME,TEXT1 FROM report_org_map"""
refer_df = pd.read_sql(sql, researchrpt_conn)

In [42]:
refer_df = refer_df.rename(columns = {"TGT_ORG_ID":"ORGAN_ID"})

In [43]:
final_df = pd.merge(merge_df, refer_df, how='left', on='ORGAN_ID') 

In [47]:
final_df = final_df.drop('ORGAN_ID', axis=1)

In [48]:
final_df.head()

,ORIGIN_ID,CODE,CODE_NAME,TITLE,AUTHOR,CREATE_DATE_x,SCORE_ID_x,CREATE_DATE_y,SCORE_ID_y,TGT_ORG_NAME,TEXT1
0,3602751,,中集安瑞科,油气勘探开发力度提升，光伏设备投产需求旺盛,"孔令鑫,张梓丁",2019-01-22,0.0,2018-10-24,0.0,中金,26455
1,3616257,000001,平安银行,四季度基金持仓分析：整体持仓小幅下滑，配置偏好业绩改善个股,王一川,2019-01-22,5.0,2019-01-13,5.0,长江证券,345
2,3602608,000002,万科A,地产12月观察：主要矛盾，和新的超预期,"谢皓宇,卜文凯,白淑媛",2019-01-22,7.0,2019-01-13,7.0,国泰君安,1786
3,3604831,000002,万科A,房地产行业2019年1月跟踪报告：难得的错配,"陈聪,张全国",2019-01-22,7.0,2019-01-03,7.0,中信证券,625
4,3604754,000002,万科A,2018年房地产投资销售保持高位运行,"高建,张云凯",2019-01-22,5.0,2019-01-07,5.0,东北证券,272


In [49]:
# # rename columns 
# REPORT_ID
# SEC_CODE
# SEC_NAME
# TITLE
# REPORT_TYPE
# ORG_ID
# ORG_NAME
# AUTHOR
# THIS_WRITE_DATE
# LAST_WRITE_DATE
# THIS_RATING
# LAST_RATING

final_df.columns = ["REPORT_ID","SEC_CODE","SEC_NAME","TITLE","AUTHOR","THIS_WRITE_DATE","THIS_RATING","LAST_WRITE_DATE","LAST_RATING","ORG_NAME","ORG_ID"]

In [50]:
# 最后添加两个不用进行表查询的列
final_df["REPORT_TYPE"] = "R03"

In [52]:
final_df["RATING_ADJUST_MARK"] = final_df[["THIS_RATING", "LAST_RATING"]].apply(compare, axis=1)

In [53]:
final_df["THIS_RATING"] = final_df["THIS_RATING"].fillna(0).astype(int)
final_df["LAST_RATING"] = final_df["LAST_RATING"].fillna(0).astype(int)

In [54]:
final_df.head()

,REPORT_ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,THIS_WRITE_DATE,THIS_RATING,LAST_WRITE_DATE,LAST_RATING,ORG_NAME,ORG_ID,REPORT_TYPE,RATING_ADJUST_MARK
0,3602751,,中集安瑞科,油气勘探开发力度提升，光伏设备投产需求旺盛,"孔令鑫,张梓丁",2019-01-22,0,2018-10-24,0,中金,26455,R03,4
1,3616257,000001,平安银行,四季度基金持仓分析：整体持仓小幅下滑，配置偏好业绩改善个股,王一川,2019-01-22,5,2019-01-13,5,长江证券,345,R03,0
2,3602608,000002,万科A,地产12月观察：主要矛盾，和新的超预期,"谢皓宇,卜文凯,白淑媛",2019-01-22,7,2019-01-13,7,国泰君安,1786,R03,0
3,3604831,000002,万科A,房地产行业2019年1月跟踪报告：难得的错配,"陈聪,张全国",2019-01-22,7,2019-01-03,7,中信证券,625,R03,0
4,3604754,000002,万科A,2018年房地产投资销售保持高位运行,"高建,张云凯",2019-01-22,5,2019-01-07,5,东北证券,272,R03,0


In [44]:
# refer_df

# 评级调整表

# R04

# 由于底层数据修正， 需要对上层数据做修正

In [6]:
sql = """SELECT ID, SEC_CODE, SEC_NAME, TITLE, AUTHOR, WRITE_DATE, RATING_ID, ORG_ID, ORG_NAME
        from rr_main where SEC_CODE=%s and ORG_ID=%s""" % ('000001','2019-03-21')
base_df = pd.read_sql(sql, researchrpt_conn)

In [7]:
base_df

,ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,WRITE_DATE,RATING_ID,ORG_ID,ORG_NAME


In [ ]:
base_df = base_df.sort_values("WRITE_DATE")

shift_df = base_df[["SEC_CODE", "ORG_ID", "WRITE_DATE", "RATING_ID"]]
last_df = shift_df.shift(1)[["WRITE_DATE", "RATING_ID"]]
last_df.columns = ['write_date_y', 'RATING_ID_y']
final_df = pd.concat([base_df, last_df], axis=1)
final_df.columns = ["REPORT_ID", "SEC_CODE", "SEC_NAME", "TITLE", "AUTHOR", "THIS_WRITE_DATE",
                    "THIS_RATING", "ORG_ID", "ORG_NAME", "LAST_WRITE_DATE", "LAST_RATING"]
# TODO dropna
final_df = final_df.dropna(subset=["THIS_RATING", "LAST_RATING"], how='any')
final_df["REPORT_TYPE"] = "R04"

In [1]:
# base_df
filter_str = "WRITE_DATE" # 根据 num 列分开1，2 后， 需要从num=1的数据中筛选出日期为查询日期的数据
this_date_str = '2019-03-21'
primary_key = ["SEC_CODE", "ORG_ID"] # 用于merge
last_key = ["WRITE_DATE", "RATING_ID"] # 同一个机构上次对本证券做的预测报告的撰写日期 同一个机构上次对本证券做的预测报告的评级

In [6]:
sql = """
SELECT ID, SEC_CODE, SEC_NAME, TITLE, AUTHOR, WRITE_DATE, RATING_ID, ORG_ID, ORG_NAME, num
FROM (
	SELECT ID, SEC_CODE, SEC_NAME, TITLE, AUTHOR, WRITE_DATE, RATING_ID, ORG_ID, ORG_NAME,
		   (@row_number:=
			   case when SEC_CODE = @pre_code and ORG_ID = @pre_organ_id
			then @row_number+1
			 else 1
			end
		   ) as num,
		(@pre_code := SEC_CODE),
		(@pre_organ_id := ORG_ID)
	FROM rr_main,(select @row_number:=1,@pre_code:='',@pre_organ_id:='') a
	where WRITE_DATE <= '%s' and report_type='R04'
	order by SEC_CODE,ORG_ID,WRITE_DATE desc
) a
where a.num<=2
""" % (this_date_str)
base_df = pd.read_sql(sql, researchrpt_conn)

In [226]:
# sql

In [7]:
base_df.tail(10)
# base_df["WRITE_DATE"] = base_df["WRITE_DATE"].astype(str)
# base_df.info()
# base_df[base_df[filter_str]=='2018-11-30']
# base_df[base_df['WRITE_DATE']=='2018-11-30']

,ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,WRITE_DATE,RATING_ID,ORG_ID,ORG_NAME,num
13363,3523998,WUBA,58同城,58同城（WUBA）：2018年三季度业绩好于预期，付费会员账户增长减慢，“收集”,罗沛达,2018-11-30,5.0,1786,国泰君安,1.0
13364,3669776,WUBA,58同城,58同城（WUBA）：虽收入短期承压，然投入仍将持续,吴越,2019-03-03,7.0,26455,中金公司,1.0
13365,3529319,WUBA,58同城,58同城（WUBA）：信息平台龙头优势明显，蓝领招聘大有可为,张忆东,2018-11-21,5.0,26548,兴业证券,1.0
13366,3675680,WUBA,58同城,58同城（WUBA）：全年完美收官符合预期，未来短期面临业绩压力,国元国际研究所,2019-03-12,5.0,31727,国元国际,1.0
13367,3671568,YY,欢聚时代,欢聚时代：海外业务拓展，整装待发,吴越,2019-03-07,7.0,26455,中金公司,1.0
13368,3532756,YY,欢聚时代,欢聚时代：携手小米直播，以期增加活跃用户,吴越,2018-11-15,7.0,26455,中金公司,2.0
13369,3533584,YY,欢聚时代,欢聚时代：付费率有所回升，但Q4指引收入增长放缓,"曹天宇,范佳瓅,秦波",2018-11-13,7.0,26504,光大证券,1.0
13370,3668910,YY,欢聚时代,欢聚时代（YY）2018Q4财报短评报告：BIGO收购完成，AI研发与全球化成为核心战略,杨仁文,2019-03-05,NaN,27302,方正证券,1.0
13371,3532872,YY,欢聚时代,欢聚时代（YY）：欢聚时代2018Q3短评报告：虎牙驱动业绩增长，新功能与内容拉升付费渗透，...,杨仁文,2018-11-14,NaN,27302,方正证券,2.0
13372,3530849,YY,欢聚时代,欢聚时代（YY）：虎牙助力营收高速增长，战略合作小米实现共赢,许磊,2018-11-15,7.0,33024,东兴证券,1.0


In [8]:
# 用num的值区分次序
last_df = base_df[base_df["num"]==2]
last_df = last_df.drop("num", 1)

In [9]:
# last_df

In [10]:
base_df[filter_str] = base_df[filter_str].astype(str)
this_df = base_df[(base_df["num"]==1) & (base_df[filter_str]==this_date_str)]
this_df = this_df.drop("num", 1)


In [11]:
# this_df

In [12]:
if len(this_df) == 0:
    # 说明没有目标数据， 可以返回
    pass


In [299]:
# primary_key.append(last_key)
primary_key
last_key
# last_df[[last_key.extend(primary_key)]]

['WRITE_DATE', 'RATING_ID', 'SEC_CODE', 'ORG_ID']

In [13]:
key = last_key + primary_key


In [25]:

merge_df = pd.merge(this_df, last_df[key], how='left', on=primary_key)

In [26]:
# merge 获取上一次最近的数据时， 若上一次数据为空， 删除它们
# merge_df = merge_df.dropna(subset=['WRITE_DATE_y'])

In [27]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 2 to 225
Data columns (total 11 columns):
ID              109 non-null int64
SEC_CODE        109 non-null object
SEC_NAME        109 non-null object
TITLE           109 non-null object
AUTHOR          109 non-null object
WRITE_DATE_x    109 non-null object
RATING_ID_x     107 non-null float64
ORG_ID          109 non-null int64
ORG_NAME        109 non-null object
WRITE_DATE_y    109 non-null object
RATING_ID_y     107 non-null float64
dtypes: float64(2), int64(2), object(7)
memory usage: 10.2+ KB


In [28]:
merge_df.head()

,ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,WRITE_DATE_x,RATING_ID_x,ORG_ID,ORG_NAME,WRITE_DATE_y,RATING_ID_y
2,3687394,00003,香港中华煤气,香港中华煤气（00003）：2018年年度业绩超出市场预期，维持“买入”评级,郭勇,2019-03-21,7.0,1786,国泰君安证券,2018-08-23,7.0
4,3682572,000401,冀东水泥,冀东水泥（000401）：京津冀水泥龙头，量价弹性持续释放,"盛昌盛,武浩翔",2019-03-21,5.0,15750,天风证券,2019-01-11,5.0
5,3683101,000401,冀东水泥,冀东水泥（000401）：京津冀龙头，受益于区域重点工程提速,"李可悦,陈彦",2019-03-21,7.0,26455,中金公司,2018-03-23,7.0
6,3683245,000401,冀东水泥,冀东水泥（000401）：重组完成新出发，关注区域基建项目推进,"黄道立,陈颖",2019-03-21,7.0,26500,国信证券,2019-02-28,7.0
7,3682449,000401,冀东水泥,冀东水泥（000401）：Q4经营略超预期，19年表现将更为靓丽,"黄诗涛,石峰源",2019-03-21,7.0,27276,国盛证券,2019-01-24,7.0


In [29]:
# merge_df.to_csv("write_date_2019-03-21.csv", index=False, encoding='utf-8')

In [30]:
# cmb_report_research.ORGAN_ID=report_org_map.TGT_ORG_ID
# sql = """SELECT distinct TGT_ORG_ID as ORG_ID,TGT_ORG_NAME,TEXT1 FROM report_org_map"""
# refer_df = pd.read_sql(sql, researchrpt_conn)

In [31]:
# refer_df.head()

In [32]:
# final_df = pd.merge(merge_df, refer_df, how='left', on='ORG_ID') 

In [33]:
# final_df = final_df.drop('ORG_ID', axis=1)

In [34]:
# final_df.head()
final_df = merge_df

In [35]:
# # rename columns 
# REPORT_ID
# SEC_CODE
# SEC_NAME
# TITLE
# REPORT_TYPE
# ORG_ID
# ORG_NAME
# AUTHOR
# THIS_WRITE_DATE
# LAST_WRITE_DATE
# THIS_RATING
# LAST_RATING

final_df.columns = ["REPORT_ID","SEC_CODE","SEC_NAME","TITLE","AUTHOR","THIS_WRITE_DATE","THIS_RATING","ORG_ID","ORG_NAME","LAST_WRITE_DATE","LAST_RATING"]

In [36]:
# 最后添加两个不用进行表查询的列
final_df["REPORT_TYPE"] = "R04"

In [37]:
# 将本次评级与上次评级做对比，取常量。0未调（两次评级结果一样）；1上调（本次大于上次）；2下调（本次小于上次）；3未知
def compare(row):
    x = row[0]
    y = row[1]
    if x == 0 and y == 0:
        return 4 # to be removed 
#     print x
    if x > y:
        return 1
    elif x < y:
        return 2
    elif x == y:
        return 0
    else:
        return 3
final_df["RATING_ADJUST_MARK"] = final_df[["THIS_RATING", "LAST_RATING"]].apply(compare, axis=1)

In [38]:
final_df["THIS_RATING"] = final_df["THIS_RATING"].fillna(0).astype(int)
final_df["LAST_RATING"] = final_df["LAST_RATING"].fillna(0).astype(int)

In [386]:
# final_df